# Crimson Responses - kilosort

basically the same thing as the other crimson response stuff, except we're working with kilosort stuff

In [2]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal
from sklearn.decomposition import PCA
import os, glob, re
import openephys_utils 
from tqdm.notebook import tqdm
import kilosort

%matplotlib qt


batch run through kilosort on filtered data

october 2nd

In [ ]:
# get a list of directories
base_dir = 'Z:\\BrainPatch\\20241002\\lateral\\'
directories = [directory for directory in os.listdir() if '2ms' in directory]

# probe and settings
probe_name = 'Z:\\BrainPatch\\20241002\\64-4shank-poly-brainpatch-chanMap.mat'
settings = {'probe_name':probe_name,
            'n_chan_bin':64,
            'nearest_chans':1}

# sos for a BPF
sos = signal.butter(N = 8, Wn = [300, 6000], btype='bandpass', fs=30000, output='sos')

for directory in tqdm(directories):
    # pull out the current and distance
    current = int(re.search('(\d{1,2})mA', directory)[1])
    distance = int(re.search('(\d{3,4})um', directory)[1])

    # print it all out
    print(f'{current}mA {distance}mm')

    # get sig_eraasr if it doesn't exist already
    if not os.path.exists(directory, 'sig_eraasr.npy'):
        sig, timestamps, stims, stim_ts = openephys_utils.open_sig_stims(os.path.join(base_dir,directory))

        # clean, pull out threshold crossings, get firing rates
        sig_eraasr = openephys_utils.ERAASR(sig, save_dir=directory)
        # spike_dict = openephys_utils.threshold_crossings(sig_eraasr, multi_rejection=None, low_cutoff=-20)
        # fr, bins = openephys_utils.calc_FR(spike_dict, max_samp = sig_eraasr.shape[0], min_samp = 0, bin_sec = bin_sec)
    else:
        sig_eraasr = np.load(os.path.join(directory, 'sig_eraasr.npy'))

    # filter at 300, 6000
    sig_filter = signal.sosfiltfilt(sos, sig_eraasr, axis=0)
    np.save(os.path.join(directory, 'sig_filter.npy'))


    # kilosort
    settings['filename'] = os.path.join(directory,'sig_eraasr.npy')
    print('kilosort on unfiltered')
    kilosort.run_kilosort(settings, file_object=sig_eraasr.astype(np.float32), data_dtype='float32', results_dir=os.path.join(directory, 'kilosort4_unfiltered'))
    settings['filename'] = os.path.join(directory,'sig_filter.npy')
    print('kilosort on filtered')
    kilosort.run_kilosort(settings, file_object=sig_eraasr.astype(np.float32), data_dtype='float32', results_dir=os.path.join(directory, 'kilosort4_filtered'))


    # spike_times = np.load(os.path.join(directory, 'sig_'))